In [1]:
import os
import time

import numpy as np
import cv2
import torch
import torch.nn.functional as F
import pytorch_lightning as pl

import hydra

from environment import CarlaEnv
from projects.symbolic_mcts.symbolic_env import SymbolicCarlaEnv
from projects.symbolic_mcts.dataset import SymbolicDataset, MLPDataset, symbolic_collate_fn
from projects.symbolic_mcts.models import TransformerAgent, MLPAgent
from environment.config.config import DefaultMainConfig
from environment.config.observation_configs import *
from environment.config.scenario_configs import *
from environment.config.action_configs import *

In [2]:
gpu = 0

In [3]:
config = DefaultMainConfig()
config.populate_config(
    observation_config = "VehicleDynamicsObstacleNoCameraConfig",
    action_config = "MergedSpeedScaledTanhConfig",
    reward_config = "Simple2RewardConfig",
    scenario_config = "NoCrashDenseTown02Config",
    testing = False,
    carla_gpu = gpu
)

# # add camera + rendering for visualization
# config.obs_config.sensors['sensor.camera.rgb/front'] = {
#     'x':2.0,
#     'z':1.4,
#     'pitch':0.0,
#     'sensor_x_res':'256',
#     'sensor_y_res':'256',
#     'fov':'120',
#     'sensor_tick': '0.0'
# }
# config.render_server = True

/zfsauton2/home/brianyan/anaconda3/envs/carla/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
env = SymbolicCarlaEnv(config=config)

  0%|          | 0/45 [00:00<?, ?it/s]

Attempting to start carla on GPU 0
Launched server at port: 56583
Waiting 45s for server to finish setting up


100%|██████████| 45/45 [00:45<00:00,  1.00s/it]


server_version 0.9.10.1-dirty


In [5]:
# agent = TransformerAgent.load_from_checkpoint('/home/scratch/brianyan/outputs/transformer/2021-09-20_10-22-58/checkpoints/epoch=6-step=88024.ckpt')
# agent.cuda().eval()

In [6]:
def save_video(frames, fname, fps=15):
    frames = [np.array(frame) for frame in frames]
    height, width = frames[0].shape[0], frames[0].shape[1]
    fourcc = cv2.VideoWriter_fourcc(*'MPEG')
    out = cv2.VideoWriter(fname, fourcc, fps, (width, height))
    for frame in frames:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        out.write(frame)
    out.release()

In [7]:
import matplotlib.pyplot as plt
from IPython import display
from datetime import datetime

# save_path = datetime.now().strftime("%m-%d-%H-%M")
# import os
# os.mkdir('/home/scratch/brianyan/outputs/evaluation_videos_' + save_path)

for i in range(25):
    frames = []
    obs = env.reset(unseen=True, index=i)
    for s in range(5000):
        # action = agent.predict(obs)
        action = env.get_autopilot_action(0.5)
        obs, reward, done, info = env.step(action)

        # frame = env.render()
        # frames.append(frame)

        if done:
            break

    print(info['termination_state'])

    # make video
    # save_video(frames, '/home/scratch/brianyan/outputs/evaluation_videos_{}/episode_{}_{}.avi'.format(save_path, i, info['termination_state']))

lane_invasion
obs_collision
lane_invasion
success
lane_invasion
lane_invasion
lane_invasion


KeyboardInterrupt: 

In [ ]:
env.close()

Killed server process
